<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/NLP_With_Deep_Learning_Prcatice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro
이것은 https://wikidocs.net/24996 의 실습입니다

# Session1 : 글자 단위의 Seq2Seq

## import dataset

In [ ]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with http.request("GET", url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

In [ ]:
lines = pd.read_csv("fra.txt", names=['src', 'tar', 'copy'], sep='\t')
len(lines)

178009

In [ ]:
lines

,src,tar,copy
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
3,Run!,Cours !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,Courez !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
...,...,...,...
178004,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç...",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
178005,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
178006,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
178007,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [ ]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]

In [ ]:
lines.sample(10)

,src,tar
23761,He took a big risk.,Il a pris un gros risque.
57305,She showed me his album.,Elle m'a montré son album.
47038,Do you believe in UFOs?,Croyez-vous aux OVNIs ?
31318,Open up the package.,Ouvrez le paquet.
38301,They're not all busy.,Elles ne sont pas toutes occupées.
8529,I'm not joking.,Je ne plaisante pas.
7316,Can I have one?,Puis-je en avoir un ?
33577,You're almost right.,Vous avez presque raison.
6001,I'm not going.,Je ne m'en vais pas.
51034,The middle one is mine.,Celui du milieu est le mien.


## Make Symbol

In [ ]:
lines.tar = lines.tar.apply(lambda x : '\t' + x + '\n')
lines.sample(10)

,src,tar
56458,I'm too sleepy to drive.,\tJe suis trop somnolent pour conduire.\n
11766,Is it hazardous?,\tEst-ce dangereux ?\n
24292,I have a dry cough.,\tJ'ai une toux sèche.\n
31722,That's a huge organ.,\tC'est un organe gigantesque.\n
22502,You have problems.,\tVous avez des problèmes.\n
24908,I'll call the cops.,\tJe vais appeler les flics.\n
1115,Be content.,\tSois satisfait !\n
6669,This is basic.,\tC'est basique.\n
33032,What's the time now?,\tQuelle heure est-il maintenant ?\n
50984,The event is on Monday.,\tL'évènement se déroule lundi.\n


## make CharSet
글자수 Set을 만들면 쉽게 중복을 안 셀수 있다...

In [ ]:
src_vocab = set()

for line in lines.src: # 1줄씩 읽기
  for char in line: # 1개씩 읽기
    src_vocab.add(char)

In [ ]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 '’',
 '€'}

In [ ]:
tar_vocab = set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [ ]:
src_vocab_size = len(src_vocab ) + 1
tar_vocab_size = len(tar_vocab ) + 1

## 글자단위 인코딩

In [ ]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

In [ ]:
src_to_index

{' ': 3,
 '!': 20,
 '"': 53,
 '$': 26,
 '%': 38,
 '&': 28,
 "'": 23,
 ',': 13,
 '-': 33,
 '.': 43,
 '/': 37,
 '0': 77,
 '1': 18,
 '2': 52,
 '3': 17,
 '4': 11,
 '5': 27,
 '6': 16,
 '7': 73,
 '8': 64,
 '9': 45,
 ':': 32,
 '?': 7,
 'A': 46,
 'B': 14,
 'C': 63,
 'D': 70,
 'E': 54,
 'F': 60,
 'G': 19,
 'H': 58,
 'I': 71,
 'J': 40,
 'K': 67,
 'L': 35,
 'M': 56,
 'N': 49,
 'O': 78,
 'P': 30,
 'Q': 5,
 'R': 24,
 'S': 75,
 'T': 2,
 'U': 10,
 'V': 68,
 'W': 57,
 'X': 34,
 'Y': 51,
 'Z': 65,
 'a': 62,
 'b': 41,
 'c': 69,
 'd': 48,
 'e': 39,
 'f': 66,
 'g': 47,
 'h': 31,
 'i': 61,
 'j': 42,
 'k': 15,
 'l': 50,
 'm': 29,
 'n': 22,
 'o': 21,
 'p': 76,
 'q': 72,
 'r': 9,
 's': 12,
 't': 44,
 'u': 25,
 'v': 36,
 'w': 8,
 'x': 59,
 'y': 4,
 'z': 55,
 'é': 1,
 '’': 6,
 '€': 74}

## 정수 인코딩

In [ ]:
encoder_input = []
for line in lines.src:
  temp_X = []
  for w in line:
    temp_X.append(src_to_index[w])
  encoder_input.append(temp_X)

print(encoder_input[:5])

[[19, 21, 43], [58, 61, 43], [58, 61, 43], [24, 25, 22, 20], [24, 25, 22, 20]]


In [ ]:
decoder_input = []
for line in lines.tar:
  temp_X = []
  for w in line:
    temp_X.append(tar_to_index[w])
  decoder_input.append(temp_X)

print(decoder_input[:5])

[[85, 22, 101, 53, 59, 4], [85, 105, 101, 43, 84, 96, 53, 59, 4], [85, 105, 101, 43, 84, 96, 95, 4], [85, 18, 34, 84, 78, 80, 72, 59, 4], [85, 18, 34, 84, 78, 93, 45, 72, 59, 4]]


## \<SOS> 제거

In [ ]:
decoder_target = []

for line in lines.tar:
  t = 0
  temp_x = []
  
  for w in line:
    if t>0:
      temp_x.append(tar_to_index[w])
    t += 1

  decoder_target.append(temp_x)

print(decoder_target[:5])

[[22, 101, 53, 59, 4], [105, 101, 43, 84, 96, 53, 59, 4], [105, 101, 43, 84, 96, 95, 4], [18, 34, 84, 78, 80, 72, 59, 4], [18, 34, 84, 78, 93, 45, 72, 59, 4]]


## 길이 맞추기

In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

print(max_src_len)
print(max_tar_len)

24
74


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding="post")
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')

decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')


## 모든 벡터를 one-hot 으로 변환

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

## 모델 설계

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]
# 은닉 상태와 게이트

In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _ , _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 79)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 106)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 344064      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  371712      input_2[0][0]                    
                                                                 lstm[0][1]            

In [ ]:
model.fit(x = [encoder_input, decoder_input], y = decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
750/750 [==============================] - 11s 15ms/step - loss: 0.7833 - val_loss: 0.6960
Epoch 2/50
750/750 [==============================] - 10s 14ms/step - loss: 0.4916 - val_loss: 0.5647
Epoch 3/50
750/750 [==============================] - 10s 14ms/step - loss: 0.4097 - val_loss: 0.5011
Epoch 4/50
750/750 [==============================] - 10s 14ms/step - loss: 0.3632 - val_loss: 0.4627
Epoch 5/50
750/750 [==============================] - 10s 14ms/step - loss: 0.3320 - val_loss: 0.4320
Epoch 6/50
750/750 [==============================] - 10s 14ms/step - loss: 0.3106 - val_loss: 0.4146
Epoch 7/50
750/750 [==============================] - 10s 14ms/step - loss: 0.2938 - val_loss: 0.4010
Epoch 8/50
750/750 [==============================] - 10s 14ms/step - loss: 0.2802 - val_loss: 0.3922
Epoch 9/50
750/750 [==============================] - 10s 14ms/step - loss: 0.2690 - val_loss: 0.3850
Epoch 10/50
750/750 [==============================] - 10s 14ms/step - loss: 0.259

## 번역기 모델의 정의

In [ ]:
# 훈련된 모델을 재사용
encoder_model = Model(inputs = encoder_inputs, outputs=encoder_states)

In [ ]:
# 이전 시점의 상태들을 저장
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
# 다음 단어를 예측하기 위해 초기 상태를 이전 시점의 상태로 사용 이는 뒤의 함수에서 구현

decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_state_inputs, outputs=[decoder_outputs] + decoder_states)


In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items()) # 코드 주의

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <sos>에 해당하는 원 - 핫 벡터 생성
  target_seq = np.zeros((1,1, tar_vocab_size))
  target_seq[0,0, tar_to_index['\t']] = 1
  
  stop_condition = False
  decoded_sentence = ""

  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예척 결과를 문자로 반환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자 추가
    decoded_sentence += sampled_char

    # <eos> 이거나 최대 길이를 넘으면 중단
    if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
      stop_condition=True
    
    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1, tar_vocab_size))
    target_seq[0,0,sampled_token_index] = 1

    # 현재 시점의 상태 저장
    states_value = [h,c]

  return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * '-')
  print("입력 문장 : ", lines.src[seq_index])
  print("정답 문장 : ", lines.tar[seq_index][1:len(lines.tar[seq_index])-1])
  print("번역기가 변역한 문장 : ", decoded_sentence[:len(decoded_sentence)-1])
  # '\n'을 빼고 출력

-----------------------------------
입력 문장 :  Run!
정답 문장 :  Cours !
번역기가 변역한 문장 :  Cours !
-----------------------------------
입력 문장 :  I left.
정답 문장 :  Je suis partie.
번역기가 변역한 문장 :  Je suis bonne.
-----------------------------------
입력 문장 :  Call us.
정답 문장 :  Appelez-nous !
번역기가 변역한 문장 :  Appelle-nous !
-----------------------------------
입력 문장 :  How nice!
정답 문장 :  Comme c'est gentil !
번역기가 변역한 문장 :  Comme c'est bonne !
-----------------------------------
입력 문장 :  Turn left.
정답 문장 :  Tourne à gauche.
번역기가 변역한 문장 :  Attrapez ça !


In [ ]:
encoder_input[3:4].shape

(1, 24, 79)

In [ ]:
encoder_input[3].shape

(24, 79)

# Session2 : 단어를 쓰는 Seq2Seq

## 파일 임포트

In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
import re
import unicodedata
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with http.request("GET", url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

In [3]:
# 이번엔 33000개의 샘플을 사용함
num_samples = 33000

## 각종 전처리 함수

In [4]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

In [5]:
def preprocess_sentence(sent):
  # 위에서 구현한 함수 호출
  sent = unicode_to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백을 만듦
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # 의도한 글자를 제외하고는 전부 공백으로 변환한다.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  sent = re.sub(r"\s+", " ", sent)

  return sent

In [6]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


In [7]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open('fra.txt', "r" ) as lines:
    for i, line in enumerate(lines):
      # 데이터 분리
      src_line, tar_line, _ = line.strip().split("\t")
      # 데이터 전처리
      src_line_input = [w for w in preprocess_sentence(src_line).split()]

      tar_line = preprocess_sentence(tar_line)
      tar_line_input = [w for w in ("<sos> " + tar_line).split()]
      tar_line_target = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line_input)
      decoder_input.append(tar_line_input)
      decoder_target.append(tar_line_target)

      if i == num_samples -1 : 
        break

    return encoder_input, decoder_input, decoder_target

In [8]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()

In [9]:
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['hi', '.'], ['hi', '.'], ['run', '!'], ['run', '!']]
[['<sos>', 'va', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.'], ['<sos>', 'cours', '!'], ['<sos>', 'courez', '!']]
[['va', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>'], ['cours', '!', '<eos>'], ['courez', '!', '<eos>']]


In [10]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

In [11]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [12]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(33000, 8)
(33000, 16)
(33000, 16)


In [13]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4663, 프랑스어 단어 집합의 크기 : 8038


In [40]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word

tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [14]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[27111 27136  9816 ... 31663 30833 26311]


In [15]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [16]:
print(encoder_input[30097])
print(decoder_input[30097])
print(decoder_target[30097])

[  2  86  59 180   1   0   0   0]
[  2  11  14 152 272   1   0   0   0   0   0   0   0   0   0   0]
[ 11  14 152 272   1   3   0   0   0   0   0   0   0   0   0   0]


In [17]:
n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [18]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [19]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(29700, 8)
(29700, 16)
(29700, 16)
(3300, 8)
(3300, 16)
(3300, 16)


## Import Seq2Seq Layer

In [20]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras import Model

In [21]:
# Embedding 크기 고정
latent_dim = 50

### 인코더 설계

In [22]:
encoder_inputs = Input(shape=(None, ))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c] # 인코더의 은닉, 셀 상태 반환

### 디코더 설계

In [23]:
decoder_inputs = Input(shape=(None, ))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states) # 인코더의 은닉 상태를 초기 상태로 사용.

decoder_dense = Dense(tar_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [24]:
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

In [25]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     233150      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     401900      input_2[0][0]                    
_______________________________________________________________________________________

In [26]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=['acc'])

In [27]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test), batch_size = 128, epochs = 50)

Epoch 1/50
233/233 [==============================] - 18s 78ms/step - loss: 3.1995 - acc: 0.6043 - val_loss: 1.9805 - val_acc: 0.7116
Epoch 2/50
233/233 [==============================] - 16s 68ms/step - loss: 1.7473 - acc: 0.7250 - val_loss: 1.6603 - val_acc: 0.7413
Epoch 3/50
233/233 [==============================] - 16s 68ms/step - loss: 1.5529 - acc: 0.7470 - val_loss: 1.5311 - val_acc: 0.7517
Epoch 4/50
233/233 [==============================] - 16s 69ms/step - loss: 1.4450 - acc: 0.7639 - val_loss: 1.4460 - val_acc: 0.7727
Epoch 5/50
233/233 [==============================] - 16s 68ms/step - loss: 1.3546 - acc: 0.7786 - val_loss: 1.3631 - val_acc: 0.7840
Epoch 6/50
233/233 [==============================] - 16s 68ms/step - loss: 1.2850 - acc: 0.7886 - val_loss: 1.3181 - val_acc: 0.7882
Epoch 7/50
233/233 [==============================] - 16s 68ms/step - loss: 1.2290 - acc: 0.7984 - val_loss: 1.2692 - val_acc: 0.7962
Epoch 8/50
233/233 [==============================] - 16s 68ms

## 번역기 동작시키기

### 변형된 모델

In [29]:
encoder_model = Model(encoder_inputs, encoder_states)

In [30]:
# decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련때 사용한 임베딩 층 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음단어 예측에 사용하기 위해 예전 상태 결과를 재사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_inputs)
decoder_state2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [33]:
# 디코더 정의
decoder_model = Model([decoder_inputs] + decoder_state_inputs,[decoder_outputs2] + decoder_state2)

### 동작을 위한 해석함수

In [42]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <sos> 생성
  target_seq = np.zeros((1,1))
  target_seq[0,0] = tar_to_index['<sos>']

  stop_condition=False
  decoded_sentence = ''

  while not stop_condition:
    # 이점 시점의 상태를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0,-1, :])
    sampled_char = index_to_tar[sampled_token_index]

    decoded_sentence += ' ' + sampled_char

    if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
      stop_condition = True

    # 현 시점의 예측 결과를 다음 시점의 입력으로 사용
    target_seq = np.zeros((1,1))
    target_seq[0,0] = sampled_token_index

    # 현 시점의 상태 저장
    states_value = [h, c]

  return decoded_sentence

In [35]:
# 원문의 정수 시퀀스를 변환
def seq2src(input_seq):
  temp = ''
  for i in input_seq:
    if(i != 0):
      temp = temp + index_to_src[i] + ' '
  return temp

In [44]:
def seq2tar(input_seq):
  temp = ''
  for i in input_seq:
    if((i != 0 and i != tar_to_index['<sos>']) and i!= tar_to_index['<eos>']):
      temp = temp + index_to_tar[i] + ' '

  return temp

### 훈련 결과 출력

In [47]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  
  print('원문 : ', seq2src(encoder_input_train[seq_index]))
  print("변역문 : ", seq2tar(decoder_input_train[seq_index]))
  print("예측문 : ", decoded_sentence[: -5]) # <eos> 떼기
  print('\n')

원문 :  you are a student . 
변역문 :  tu es un etudiant . 
예측문 :   tu es un nouveau . 


원문 :  i deserve it . 
변역문 :  je le merite . 
예측문 :   je le coup . 


원문 :  let s hit the road . 
변역문 :  on se casse ! 
예측문 :   nous faites une journee . 


원문 :  we re fighting . 
변역문 :  nous sommes en train de nous battre . 
예측문 :   nous nous en train de ne nous faut pas . 


원문 :  i was happy then . 
변역문 :  j etais heureux alors . 
예측문 :   j etais heureux pour vous . 


